In [128]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, DateTime, Text, ForeignKey
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

In [129]:
travel_sensor_df = pd.read_csv("travel_sensor.csv")
sensor_data_df = pd.read_csv("sensor_data.csv")
traffic_detectors_df = pd.read_csv("traffic_detectors.csv")

In [130]:
engine = create_engine(f"sqlite:///austin_traffic.sqlite")
conn = engine.connect()

In [131]:
class Detector(Base):
    __tablename__ = 'traffic_detector'
    __table_args__ = {'extend_existing': True} 
    index = Column(Integer(), primary_key=True)
    atd_location_id = Column(String(20))
    created_date = Column(String(25))
    detector_id = Column(Integer())
    detector_status = Column(String(20))
    detector_type = Column(String(20))
    ip_comm_status = Column(String(20))
    location_latitude = Column(Integer())
    location_longitude = Column(Integer())
    location_name = Column(String(50))
    modified_date = Column(String(25))
    signal_id = Column(Integer())

In [132]:
class Data(Base):
    __tablename__ = 'sensor_data'
    index = Column(Integer(), primary_key=True)
    curdatetime = Column(Integer())
    day = Column(Integer())
    day_of_week = Column(Integer())
    detid = Column(Integer())
    detname = Column(String(20))
    direction = Column(String(20))
    hour = Column(Integer())
    int_id = Column(Integer())
    intname = Column(String(20))
    minute = Column(Integer())
    month = Column(Integer())
    occupancy = Column(Float)
    row_id = Column(String(50))
    speed = Column(Float)
    timebin = Column(String(20))
    volume = Column(Float)
    year = Column(Integer())

In [133]:
class Sensor(Base):
    __tablename__ = 'travel_sensor'
    index = Column(Integer(), primary_key=True)
    atd_location_id = Column(String(20))
    atd_sensor_id = Column(Integer())
    coa_intersection_id = Column(Integer())
    comm_status_datetime_utc = Column(String(50))
    council_district = Column(Integer())
    cross_st = Column(String(20))
    cross_st_block = Column(Integer())
    ip_comm_status = Column(String(20))
    kits_id = Column(Integer())
    landmark = Column(String(20))
    location = Column(String(70))
    location_latitude = Column(Integer())
    location_longitude = Column(Integer())
    location_name = Column(String(50))
    location_type = Column(String(20))
    modified_date = Column(String(25))
    primary_st = Column(String(20))
    primary_st_aka = Column(String(20))
    primary_st_block = Column(Integer())
    reader_id = Column(Integer())
    sensor_mfg = Column(String(20))
    sensor_status = Column(String(20))
    sensor_type = Column(String(20))
    turn_on_date = Column(String(25))

In [134]:
Base.metadata.create_all(engine)

In [135]:
travel_sensor_data = travel_sensor_df.to_dict(orient="records")
sensor_data_data = sensor_data_df.to_dict(orient="records")
traffic_detectors_data = traffic_detectors_df.to_dict(orient="records")

In [136]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = sqlalchemy.schema.MetaData(bind=engine)
metadata.reflect()

In [137]:
travel_sensor_table = sqlalchemy.Table('travel_sensor', metadata, autoload=True)

conn.execute(travel_sensor_table.delete())

conn.execute(travel_sensor_table.insert(), travel_sensor_data)

In [138]:
sensor_data_table = sqlalchemy.Table('sensor_data', metadata, autoload=True)

conn.execute(sensor_data_table.delete())

conn.execute(sensor_data_table.insert(), sensor_data_data)

In [139]:
traffic_detectors_table = sqlalchemy.Table('traffic_detector', metadata, autoload=True)

conn.execute(traffic_detectors_table.delete())

conn.execute(traffic_detectors_table.insert(), traffic_detectors_data)

In [140]:
engine.execute("select * from traffic_detector limit 2").fetchall()

[(1, 'LOC16-004875', '2017-06-28T05:00:00', 1550, 'OK', 'LOOP', None, 30.4469604, -97.7977066, '9700 BLK ANDERSON MILL RD (Bethany)', '1498626000', 997),
 (2, 'LOC16-004875', '2017-06-28T05:00:00', 1551, 'OK', 'LOOP', None, 30.4469604, -97.7977066, '9700 BLK ANDERSON MILL RD (Bethany)', '1498626000', 997)]

In [141]:
engine.execute("select * from sensor_data limit 2").fetchall()

[(1, 1534119301, 12, 0, 87, 'SB_out', 'SB', 19, 23, 'BURNETPALM WAY', 15, 8, 2.0, '1abed4f17c5a1a00ea9a49067e7e4d9a', 37.0, '19:15', 36.0, 2018),
 (2, 1534119301, 12, 0, 88, 'SB_in', 'SB', 19, 23, 'BURNETPALM WAY', 15, 8, 2.0, '05234c341297c5bbf669b012176bc9d5', 35.0, '19:15', 49.0, 2018)]

In [142]:
engine.execute("select * from travel_sensor limit 2").fetchall()

[(1, 'LOC17-010425', 158, None, '2018-03-29T03:35:00.000', 9, '  ', None, 'ONLINE', 20, None, "{'type': 'Point', 'coordinates': [-97.737262, 30.261476]}", 30.261476000000002, -97.737262, '700 BLK E CESAR CHAVEZ ST', 'ROADWAY', '2017-07-03T11:04:00.000', ' CESAR CHAVEZ ST', None, 700, None, 'Wavetronix', 'TURNED_ON', 'RADAR', '2015-10-15T00:00:00.000'),
 (2, 'LOC17-010430', 159, 5155709, '2018-03-29T03:35:00.000', 9, ' JOHANNA ST', 101, 'ONLINE', 17, 'Fulmore Middle School', "{'type': 'Point', 'coordinates': [-97.751737, 30.244513]}", 30.244513, -97.751737, ' CONGRESS AVE / JOHANNA ST (Fulmore Middle School)', 'ROADWAY', '2017-07-03T11:15:00.000', ' CONGRESS AVE', None, 1900, None, 'Wavetronix', 'TURNED_ON', 'RADAR', '2016-04-04T00:00:00.000')]